## Prueba BOHB

In [1]:
# Creación de datos sintéticos para clasificación binaria

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd

# Generamos un data set de 500 filas y 10 características
X, y = make_classification(
    n_samples=500,   # Número de filas
    n_features=10,   # Número de características
    n_informative=8, # Características relevantes
    n_redundant=2,   # Características redundantes
    random_state=42
)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
X_train[:5], y_train[:5]

(array([[ 1.82325985, -0.2912727 ,  0.43310216,  2.25296944, -1.54561288,
         -2.03217427, -0.314744  ,  0.29454587, -0.47873915,  2.38427661],
        [ 1.99534595, -1.35912957, -0.38404344, -1.661742  ,  4.17523872,
         -3.29599262, -2.06171655,  2.78381625,  1.7601656 ,  0.53338158],
        [ 1.40151215, -2.04928361, -1.47135028,  3.71183278, -0.51438725,
          0.94192995, -2.5435297 ,  1.66618656, -3.14315027,  0.93308393],
        [ 0.10828665, -0.80542805, -1.18899805, -0.44444324, -0.91517226,
         -0.52312016,  0.11663634,  2.4248636 , -0.83505002, -0.44668768],
        [-1.73758971, -3.81249015, -4.78215553,  6.05777859, -0.8998485 ,
         -2.35360688, -2.76674578,  1.49359229, -1.97799393, -0.3088274 ]]),
 array([1, 1, 0, 1, 1]))

In [5]:
import hpbandster
import ConfigSpace
print("ConfigSpace y HpBandSter instalados correctamente")

ConfigSpace y HpBandSter instalados correctamente


In [6]:
# Configuración BOHB para prueba rápida

from ray import tune
from ray.tune.schedulers import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.tune.integration.keras import TuneReportCallback
import tensorflow as tf

# Definir espacio de búsqueda reducido
search_space = {
    "num_layers": tune.randint(1, 4),  # Reducido a 1-2 capas
    "units_per_layer": tune.qrandint(10, 20, 5),  # Reducido a 32-64 neuronas
    "learning_rate": tune.loguniform(1e-3, 1e-2),  # Tasa de aprendizaje más restringida
    "batch_size": tune.choice([16, 32]),  # Solo dos opciones
    "dropout": tune.uniform(0.1, 0.3),  # Dropout entre 10%-30%
    "activation": tune.choice(["relu", "tanh"])  # Dos funciones de activación
}

# Crear y entrenar modelo
def train_model(config):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(10,)))
    
    # Agregar capas según el config
    for _ in range(config["num_layers"]):
        model.add(tf.keras.layers.Dense(
            units=config["units_per_layer"],
            activation=config["activation"]
        ))
    model.add(tf.keras.layers.Dropout(config["dropout"]))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))  # Clasificación binaria

    # Compilar modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["learning_rate"]),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    # Entrenar modelo
    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=config["batch_size"],
        epochs=10,  # Reducido para prueba rápida
        verbose=0,
        callbacks=[TuneReportCallback({"val_accuracy": "val_accuracy"})]
    )

In [7]:
# Configurar BOHB con un número limitado de pruebas

bohb_search = TuneBOHB()
bohb_scheduler = HyperBandForBOHB(max_t=10)

tune.run(
    train_model,
    config=search_space,
    metric="val_accuracy",
    mode="max",
    num_samples=10,  # Realizar solo 10 pruebas
    scheduler=bohb_scheduler,
    search_alg=bohb_search
)

OSError: [Errno 0] AssignProcessToJobObject() failed